In [1]:
import numpy as np 
import matplotlib.pyplot as plt

# Convex Optimization - Homework 3

The goal is to : $\text{minimize } \frac{1}{2} || X w - y || _2 ^2 + \lambda || w || _2 ^2 $ (LASSO) 

We've seen in question 1 that the dual problem of LASSO is : $\text{minimize } \left[ v ^T Q v + p ^T v \right] \text{   subject to   } A v \preceq b$

## Question 2 : Implement the barrier method to solve QP.

#### 2.1 Write a function **$\text{Centering Step}(Q,p,A,b,t,v_0, \epsilon )$**

> This functions implements the **Newton method** to solve the *centering step* given the inputs $(Q, p, A, b)$, the *barrier method* parameter $t$ (see lectures), initial variable $v_0$ and a target precision $\epsilon$. The function outputs the sequence of variables iterates $(v_i)_{i=1,...,n_{\epsilon}}$, where nε is the number of iterations to obtain the ε precision. Use a backtracking line search with appropriate parameters.

In [4]:
# Define all the functions needed for the barrier method
def centering_step(Q, p, A, b, v0, t, epsilon, alpha, beta):
    """
    This function performs the centering step of the barrier method

    Parameters 
    ----------
    Q : numpy array
        Quadratic term of the target function
    p : numpy array
        Linear term of the target function
    A : numpy array
        Matrix of the constraints
    b : numpy array
        Vector of the constraints
    v0 : numpy array
        Initial point
    t : float
        Parameter of the barrier method
    epsilon : float
        Precision of the centering step
    alpha : float
        Parameter of the backtracking line search algorithm
    beta : float
        Parameter of the backtracking line search algorithm
    
    Returns 
    -------
    v : numpy array
        Suit of point of the centering step
    """

    ##### First we define some usefull functions for the centering step #####   
    def target_function(Q, p):
        """ 
        This function returns the target function of the quadratic programming problem
        """
        f = lambda v : np.transpose(v) @ Q @ v + np.transpose(p) @ v
        return f

    def gradient_target(Q, p):
        """
        This function returns the gradient of the target function
        """
        grad = lambda v : 2 * Q @ v + p
        return grad

    def hessian_target(Q):
        """
        This function returns the hessian of the target function
        """
        hess = lambda v : 2 * Q
        return hess

    def phi_function(A, b):
        """
        This function returns the phi function of the quadratic programming problem
        """
        A = np.array(A)
        phi = lambda v : - np.sum(np.log(- np.dot(A, v) - b))
        return phi

    def gradient_phi(A, b):
        """
        This function returns the gradient of the phi function
        """
        A = np.array(A)
        grad = lambda v : - np.sum(A / (np.dot(A, v) - b), axis=0)
        return grad

    def hessian_phi(A, b):
        """
        This function returns the hessian of the phi function
        """
        A = np.array(A)
        hess = lambda v : np.sum(np.array([np.outer(A[i], A[i]) / (np.dot(A[i], v) - b[i]) ** 2 for i in range(len(A))]), axis=0)
        return hess

    def BM_function(f, phi):
        """
        This function returns the barrier method function
        """
        BM = lambda v, t : f(v) + t * phi(v)
        return BM

    def gradient_BM(grad_f, grad_phi, t):
        """
        This function returns the gradient of the barrier method function
        """
        grad_BM = lambda v : grad_f(v) + t * grad_phi(v)
        return grad_BM

    def hessian_BM(hess_f, hess_phi, t):
        """
        This function returns the hessian of the barrier method function
        """
        hess_BM = lambda v : hess_f(v) + t * hess_phi(v)
        return hess_BM
    ##### ----------------------------------------------------------------- #####

    # Initialize the centering step
    f = target_function(Q, p)
    grad_f = gradient_target(Q, p)
    hess_f = hessian_target(Q)
    phi = phi_function(A, b)
    grad_phi = gradient_phi(A, b)
    hess_phi = hessian_phi(A, b)
    complete_BM = BM_function(f, phi)
    BM = lambda v : complete_BM(v, t)
    grad_BM = gradient_BM(grad_f, grad_phi, t)
    hess_BM = hessian_BM(hess_f, hess_phi, t)

    # Then, we define the backtracking line search algorithm
    def backtrack_line_search(function, grad_function, v):
        """ 
        This function implements the backtracking line search algorithm

        Parameters
        ----------
        function : function to minimize
        grad_function : gradient of the function to minimize
        v : current point
        alpha : parameter of the backtracking line search algorithm
        beta : parameter of the backtracking line search algorithm
        """
        eta = 1 
        while function(v - eta * grad_function(v)) > function(v) - alpha * eta * np.dot(grad_function(v), grad_function(v)):
            eta *= beta 
        return eta
    
    # Finally, we implement the Newton's method
    liste_v = [v0]
    v = v0
    while np.linalg.norm(grad_BM(v)) > epsilon:
         delta = - grad_BM(v)
         eta = backtrack_line_search(BM, grad_BM, v, alpha, beta)
         v += eta * delta 
         liste_v.append(v)
    return liste_v

#### 2.2 Write a function **$ \text{Barr Method}(Q, p, A, b, v_0, \epsilon )$**

> This functions implements the **barrier method** to solve QP using precedent function given the data inputs $(Q, p, A, b)$, a feasible point $v0 $, a precision criterion $\epsilon $. The function outputs the sequence of variables iterates $ (v_i) _{i = 1, ..., n_{\epsilon } } $, where $n_{\epsilon }$ is the number of iterations to obtain the $ \epsilon $ precision.

In [ ]:
def Barrier_Method(Q, p, A, b, x0, eps, mu, alpha = 0.25, beta = 0.5, t = 1):
    """ 
    This function implements the barrier method
    
    Parameters
    ----------
    
    Q : numpy array
        Quadratic term of the target function
    p : numpy array
        Linear term of the target function
    A : numpy array
        Matrix of the constraints
    b : numpy array
        Vector of the constraints
    x0 : numpy array
        Initial point
    eps : float
        Precision of the barrier method
    mu : float
        Parameter of the barrier method
    alpha : float
        Parameter of the backtracking line search algorithm
    beta : float
        Parameter of the backtracking line search algorithm
    t : float   
        Parameter of the barrier method
    """
    m = len(b)    # Number of constraints
    liste_x = [x0]
    xt = x0
    while (m / t) > eps:
        xt = centering_step(Q, p, A, b, xt, t, eps, alpha, beta)
        t *= mu
        liste_x.append(xt)
    return liste_x

In [ ]:
n = 10 ** 2
X, y = 
Q = (1 / 2) * np.eye(n) 
